# Quick runthrough of what Transformers can do
This is based on code modified from the [repository](https://github.com/nlp-with-transformers/notebooks) associated with the excellent book [Natural Language Processing with Transformers](https://www.oreilly.com/library/view/natural-language-processing/9781098136789/) which I highly recommend as one of the books to have on your shelf if you are going to build a model using the Hugging Face Hub (or if you are learning how to use Transformers at all). There are other Hubs, but I find that Hugging Face is the best I've seen by far from an instructional and support perspective.

## Set up on Colab
I encourage you to set this up on Colab and your Google Drive and play with this yourself. I will be going into some of these topics in more depth during the course.

In [1]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git

Cloning into 'notebooks'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 526 (delta 143), reused 135 (delta 126), pack-reused 354
Receiving objects: 100% (526/526), 28.62 MiB | 16.31 MiB/s, done.
Resolving deltas: 100% (250/250), done.


In [2]:
%cd notebooks
!wget -O install.py "https://raw.githubusercontent.com/EMGrua/MN5162-extra/main/install.py"
!wget -O requirements.txt "https://raw.githubusercontent.com/EMGrua/MN5162-extra/main/requirements.txt"

/content/notebooks
--2024-03-05 14:30:51--  https://raw.githubusercontent.com/EMGrua/MN5162-extra/main/install.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3527 (3.4K) [text/plain]
Saving to: ‘install.py’

install.py          100%[===================>]   3.44K  --.-KB/s    in 0s      

2024-03-05 14:30:52 (57.6 MB/s) - ‘install.py’ saved [3527/3527]

--2024-03-05 14:30:52--  https://raw.githubusercontent.com/EMGrua/MN5162-extra/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 486 [text/plain]
Saving to: 

In [3]:
from install import *
install_requirements()

⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


***NOTE*** Make sure that datasets is v2.15.0 ***END NOTE***

In [4]:
#hide
from utils import *
setup_chapter()

Using transformers v4.16.2
Using datasets v2.15.0


## A Tour of Transformer Applications

In this section, I'll go through several NLU applications that Transformers can provide. There are two points in particular that I would like you to bear in mind:
* How easy the code is to set up compared to the previous "under the hood" work you may have done in order to learn deep learning for NLP
* The high level of performance even without tweaking

Every natural language processing task begines with text. The example text that is provided below is customer feedback on an online order. Those of you who are familiiar with the movie franchise "Transformers" can facepalm at the humour of the authors of [Natural Language Processing with Transformers](https://www.oreilly.com/library/view/natural-language-processing/9781098136789/)....

In [6]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

Your application might be handling a legal contract or pretty much any consequential text you can think of (and in a different lanugage too), but lets play with this example.

### Text Classification

Firstly, we would like to understand if the feedback from the customer is positive or negative, as this will determine how we process the feedback. This is where you'll see a Hugging Face Transformers pipeline in action, whereby we set up a **pipeline** for the desired **task**.

In [7]:
#hide_output
from transformers import pipeline

classifier = pipeline("text-classification")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Now we just feed the classifier our text and get the corresponding output.

In [8]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


This API approach greatly simplifies the NLU task from the perspective of the system developer and provides a good performance even before any tuning is considered.
When you first run this code, downloading of model weights from the [Hugging Face Hub](https://oreil.ly/zLK11) will take place, but for subsequent instantiations of the pipeline, the library will notice you have cached the model weights and will use those instead. Also, the **sentiment analysis** pipeline here doesn't just detect positive and negative sentiments, but can also pick up other sentiments such as joy, sadness, love, anger, fear and surprise.

In the case of this example, we can see that the sentiment of the text is, confidently, NEGATIVE.

### Named Entity Recognition

Now that we have an understanding of the feedback from the customer, but we need to extracting more information. In NLP, real-world objects, like persons, places and things are called **named entities** and extracting them from text is call **Named Entity Recognition(NER)**. We can create a named entity recognition pipeline and feed our customer feedback to it.

In [9]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556570,Mega,208,212
4,PER,0.590257,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498350,##icons,259,264
7,MISC,0.775361,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


You can see that the pipeline detected the entities and assigned them corresponding tags. It identified the word **Amazon** as an **ORG** (organisation), **Germany** as a **LOC** (location) and **Bumblebee** as a **PER** (person). Several of the tokens were not correctly identified, which suggests it would be useful to revisit the tokenizer.

### Question Answering

In question answering, a pipleine is provided with a passage of text that we want to question, called the **context**, along with a **question** about the context and the model returns the span of text corresponding to the answer. So, let's ask a reasonable question about the customer feedback.

In [10]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


### Summarization

Text summarization is intended to take a long text as input and to return a shorter version with all relevant facts; this involves generating coherent text.

In [11]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

 Bumblebee ordered an Optimus Prime action figure from your online store in
Germany. Unfortunately, when I opened the package, I discovered to my horror
that I had been sent an action figure of Megatron instead.


### Translation

It could be that you want to work between different lanugages and this can be realized with a translation pipeline.

In [12]:
translator = pipeline("translation_en_to_de",
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/750k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus
Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete,
entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von
Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich
hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere
einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt.
Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von
Ihnen zu hören. Aufrichtig, Bumblebee.


### Text Generation

Text generation has become particularly popular lately with the advent of large language models. So, in this case, we ask the system to generate a response to the customer.

In [13]:
#hide
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

In [14]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Dear Amazon, last week I ordered an Optimus Prime action figure from your online
store in Germany. Unfortunately, when I opened the package, I discovered to my
horror that I had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the
issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear
from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. I am satisfied
with your purchase. I will gladly see your product and the packaging again
before you go out buying at your shop! I'm a small business owner and have not
been able to deliver my product to customer before and after an Amazon purchase,
so I apologize.

Customer service response:

All of my orders arrived


You can see that the response is plausible, but may not something that you would want to send in that form without further edits,.... however it provides a very good automatic basis and the response, as with all responses above, are without tweaking.

So you can see the power of Transformer architectures and facility of using a model pipeline.